# 집단별 요약하기 (groupby(key).agg(sum_data = 'data','sum))
- 집단별 평균, 집단별 빈도, 집단별 합계 등과 같이 집단별 요약이 필요한 경우 사용
- 집단간 요약표를 통해 집단간 비교가 쉽다.

## Lab 1:집단별 요약하기
요약 통계량 구하기
| 함수     | 통계량     |
|----------|------------|
| mean()   | 평균       |
| std()    | 표준편차   |
| sum()    | 합         |
| median() | 중앙값     |
| min()    | 최소값     |
| max()    | 최대값     |
| count()  | 빈도(개수) |

In [1]:
import pandas as pd
exam = pd.read_csv('exam.csv')
exam.head()

,id,nclass,math,english,science
0,1,1,50,98,50
1,2,1,60,97,60
2,3,1,45,86,78
3,4,1,30,98,58
4,5,2,25,80,65


집단별 요약 통계량 구하기groupby()
1. 반별 수학 점수 평균
2. exam.groupby('nclass'): 반별로 집단을 나누고
3. exam.groupby('nclass').agg(mean_math = ('math','mean')) : 집단별로 mean_math 이름으로 math 변수에 대해 mean(평균) 산출

In [2]:
exam.groupby('nclass').agg(mean_math = ('math','mean'))

,mean_math
nclass,
1,46.25
2,61.25
3,45.00
4,56.75
5,78.00


In [3]:
#as_index = False : nclass를 인덱스로 사용하지 않음
exam.groupby('nclass', as_index = False).agg(mean_math = ('math','mean'))

,nclass,mean_math
0,1,46.25
1,2,61.25
2,3,45.00
3,4,56.75
4,5,78.00


In [4]:
#집단별 여러 통계량 한꺼번에 구하기
exam.groupby('nclass').agg(mean_math = ('math','mean'),
                           sum_math=('math','sum'),
                          median_math = ('math','median'),
                          n = ('math','count'))

,mean_math,sum_math,median_math,n
nclass,,,,
1,46.25,185,47.5,4
2,61.25,245,65.0,4
3,45.00,180,47.5,4
4,56.75,227,53.0,4
5,78.00,312,79.0,4


In [5]:
#모든 변수 통계량 한번에 구하기
exam.groupby('nclass').mean()

,id,math,english,science
nclass,,,,
1,2.5,46.25,94.75,61.50
2,6.5,61.25,84.25,58.25
3,10.5,45.00,86.50,39.25
4,14.5,56.75,84.75,55.00
5,18.5,78.00,74.25,83.25


## Lab 2:집단별로 다시 나누기
- 여러 변수를 지정하여 집단을 나누고 다시 하위 변수로 나눌 수 있다.
- mpg.groupby('manufacturer','drv']).agg(mean_cty=('cty','mean')) : 제조사(manufacturer)와 구동방식(drv)로 집단을 나누고, 집단별로 cty의 평균을 구하여 변수 mean_cty에 저장

In [6]:
mpg = pd.read_csv('mpg.csv')
mpg.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact


In [7]:
mpg.groupby(['manufacturer','drv']).agg(mean_cty=('cty','mean'))

mean_cty
manufacturer drv           
audi         4    16.818182
             f    18.857143
chevrolet    4    12.500000
             f    18.800000
             r    14.100000
dodge        4    12.000000
             f    15.818182
ford         4    13.307692
             r    14.750000
honda        f    24.444444
hyundai      f    18.642857
jeep         4    13.500000
land rover   4    11.500000
lincoln      r    11.333333
mercury      4    13.250000
nissan       4    13.750000
             f    20.000000
pontiac      f    17.000000
subaru       4    19.285714
toyota       4    14.933333
             f    21.368421
volkswagen   f    20.925926

In [8]:
#제조사 == audi인 자동차를 대상으로 drv에 따라 그룹화하고, 그룹별로
# n 변수에 구동방식 drv에 대한 그룹네 개수 값을 구하여 출력
mpg.query('manufacturer == "audi"').groupby(['drv']).agg(n=('drv','count'))

,n
drv,
4,11
f,7


## Lab 3:집단별 빈도수 구하기 value_counts()

In [9]:
# 구동방식에 따라 그룹화하고, 
# 그룹별로 n변수에 구동방식에 대한 그룹네 개수값을 구하여 출력
mpg.groupby('drv').agg(n=('drv','count'))

,n
drv,
4,103
f,106
r,25


In [13]:
# 구동방식에 대한 그룹네 개수 값을 구하여 출력
mpg.value_counts('drv') #시리즈로 출력

drv
f    106
4    103
r     25
dtype: int64

In [14]:
 mpg.value_counts('drv').to_frame('count') #데이터프레임으로 출력

,count
drv,
f,106
4,103
r,25


In [15]:
# 구동방식에 대한 그룹네 개수가 100개 초과인 구동방식을 구하여 출력
mpg.value_counts('drv').to_frame('count').query('count > 100')
# 시리즈 데이터 타입에는 query() 함수를 사용할 수 없다.

,count
drv,
f,106
4,103


## Lab 4:mpg 데이터에서 suv 제조사의 합산 평균 연비 상위 1~5위 제조사 출력하기
1. suv 추출 : query()
2. 합산 연비 변수 만들기 : assign()
3. 회사별 분리: groupby()
4. 합산 연비 평균 구하기 : agg()
5. 내림차순 정렬하기 : sort_values()
6. 1~5위까지 출력하기 : head()

In [17]:
mpg = pd.read_csv('mpg.csv')
mpg.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact


In [18]:
# mpg에서 차종(category)가 suv인 자동차만 골라 mpg_new로 생성
mpg_new = mpg.query('category == "suv"')
mpg_new.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
18,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,14,20,r,suv
19,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,11,15,e,suv
20,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,14,20,r,suv


In [19]:
# 도시연비(cty)와 고속도로연비(hwy)의 평균 값을 갖는 변수 통합연비total 만들기
mpg_new = mpg_new.assign(total = (mpg['hwy']+mpg['cty'])/2)
mpg_new.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category,total
18,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,14,20,r,suv,17.0
19,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,11,15,e,suv,13.0
20,chevrolet,c1500 suburban 2wd,5.3,2008,8,auto(l4),r,14,20,r,suv,17.0


In [20]:
# 제조사에 따라 그룹화하고,
# 그룹별로 mean_tot 변수에 통합연비 total에 대한 평균 값을 구하여 저장
mpg_new = mpg_new.groupby('manufacturer').agg(mean_tot=('total','mean'))
mpg_new.head(3)

,mean_tot
manufacturer,
chevrolet,14.888889
dodge,13.928571
ford,15.333333


In [23]:
# mpg_new를 평균 연비 변수 mean_tot 값을 기준으로 내림차순으로 정렬
mpg_new = mpg_new.sort_values('mean_tot',ascending=False)
mpg_new.head(5)

,mean_tot
manufacturer,
subaru,21.916667
toyota,16.312500
nissan,15.875000
mercury,15.625000
jeep,15.562500


In [27]:
# 상의 cell의 작업을 하나의 구문으로 처리
mpg.query('category=="suv"').assign(total=(mpg['hwy']+mpg['cty'])/2)\
.groupby('manufacturer')\
.agg(mean_tot = ('total','mean'))\
.sort_values('mean_tot',ascending=False).head(5)

,mean_tot
manufacturer,
subaru,21.916667
toyota,16.312500
nissan,15.875000
mercury,15.625000
jeep,15.562500


## Lab 5:혼자서 해보기(mpg 데이터 분석)

Q1: category별 도시 연비(cty) 평균을 구해보시오

In [3]:
import pandas as pd
mpg=pd.read_csv('mpg.csv')
mpg.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact


Q2: category별 평균 도시연비(cty)가 높은 기준으로 정렬하시오

In [10]:
mpg2= mpg.groupby('category').agg(mean_c=("cty","mean"))

Q3: 자동차의 평균 고속도로 연비(hwy)가 높은 3개 회사 출력하시오

Q4: compact 차종을 가장 많이 생산하는 회사를 출력하시오